# Import modules

In [ ]:
import warnings
import numpy as np
from keras.layers import Input, Dense, Lambda
from keras.layers.merge import concatenate as concat
from keras.models import Model
from keras import backend as K
from keras import losses, callbacks, metrics
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from scipy.misc import imsave
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
from sklearn.model_selection import train_test_split
import win32api
import random
from susyai13tev import susyai
import pickle
import seaborn as sns
from datetime import datetime
from matplotlib.colors import LogNorm

# Import data

In [ ]:
raw_data=pd.read_csv("data.csv")
data=raw_data.values
index=data[:,0]
truth_13tev=data[:,len(data[0,:])-1]
'''remove first and last column'''
data=np.delete(data,0,axis=1)
data=np.delete(data,len(data[0,:])-1,axis=1)

raw_data_label_extended=pd.read_csv("data_label_extended.csv")
data_label_extended=raw_data_label_extended.values
data_label_extended=np.delete(data_label_extended,0,axis=1)
truth_13tev = to_categorical(truth_13tev)
data_label_extended=np.concatenate([data_label_extended, truth_13tev],axis=1)

'''normalize'''
for i in range(len(data[0,:])):
    maximum=np.max(data[:,i])
    minimum=np.min(data[:,i])
    data[:,i]=(data[:,i]-minimum)/(maximum-minimum)

for i in range(len(data_label_extended[0,:])):
    maximum=np.max(data_label_extended[:,i])
    minimum=np.min(data_label_extended[:,i])
    data_label_extended[:,i]=(data_label_extended[:,i]-minimum)/(maximum-minimum)
'''plots histograms if needed''' 
#for i in range(len(data[0,:])):
#    plt.figure(i)
#    plt.hist(data[:,i],100)
#    plt.title(list(raw_data)[i+1])

x_train, x_test, y_train, y_test = train_test_split(data,data_label_extended, train_size=200000, test_size=100000)
train = np.concatenate([x_train, y_train],axis=1)
test = np.concatenate([x_test, y_test],axis=1)

# Create Model

In [ ]:
def train(epochs,min_dim,hidden_layer_size,beta):
    global cvae
    global history
    global decoder
    global encoder
    m = 1000 # batch size
    n_z = min_dim # latent space size
    encoder_dim1 = hidden_layer_size # dim of encoder hidden layer
    decoder_dim = hidden_layer_size # dim of decoder hidden layer
    decoder_out_dim = 19 # dim of decoder output layer
    activ = 'relu'
    optim = Adam(lr=0.001)
    n_x = x_train.shape[1]
    n_y = y_train.shape[1]
    n_epoch = epochs

    X = Input(shape=(n_x,))
    data_label = Input(shape=(n_y,))
    inputs = concat([X, data_label])
    
    encoder_1=Dense(encoder_dim1, activation=activ)(inputs)
    encoder_2=Dense(encoder_dim1, activation=activ)(encoder_1)
    encoder_3 = Dense(encoder_dim1, activation=activ)(encoder_2)
    mu = Dense(n_z, activation='linear')(encoder_3)
    l_sigma = Dense(n_z, activation='linear')(encoder_3)

    def sample_z(args):
        mu, l_sigma = args
        eps = K.random_normal(shape=(m, n_z), mean=0, stddev=1)
        return mu + K.exp(l_sigma / 2) * eps

    # Sampling latent space
    z = Lambda(sample_z, output_shape = (n_z, ))([mu, l_sigma])

    # merge latent space with label
    zc = concat([z, data_label])

    decoder_1 = Dense(decoder_dim, activation=activ)(zc)
    decoder_2 = Dense(decoder_dim, activation=activ)(decoder_1)
    decoder_3 = Dense(decoder_dim, activation=activ)(decoder_2)
    decoder_out = Dense(decoder_out_dim, activation='sigmoid')
    #h_p = decoder_hidden(zc)
    outputs = decoder_out(decoder_3)

    def vae_loss(y_true, y_pred):
        recon = K.sum(losses.mean_squared_error(y_true, y_pred), axis=-1)
        kl = 0.5 * K.sum(K.exp(l_sigma) + K.square(mu) - 1. - l_sigma, axis=-1)
        return recon + beta*kl

    def KL_loss(y_true, y_pred):
        return(0.5 * K.sum(K.exp(l_sigma) + K.square(mu) - 1. - l_sigma, axis=1))

    def recon_loss(y_true, y_pred):
        return K.sum(losses.mean_squared_error(y_true, y_pred), axis=-1)

    cvae = Model([X, data_label], outputs)
    encoder = Model([X, data_label], mu)

    d_in = Input(shape=(n_z+n_y,))
    d_h_1 = Dense(decoder_dim, activation=activ)(d_in)
    d_h_2 = Dense(decoder_dim, activation=activ)(d_h_1)
    d_h_3 = Dense(decoder_dim, activation=activ)(d_h_2)
    d_out = decoder_out(d_h_3)
    decoder = Model(d_in, d_out)

    cvae.compile(optimizer=optim, loss=vae_loss, metrics = [KL_loss, recon_loss])
    
    earlystop=callbacks.EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=1000,verbose=0, mode='auto')
    filepath='C:\\Users\\chrisje\\stage\\best_extended'+str(min_dim)+'.hdf5'
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', 
                                           verbose=0, save_best_only=True, save_weights_only=True)
    history=cvae.fit([x_train,y_train],x_train,
            epochs=n_epoch, verbose=2,
            batch_size=m,
            validation_data=([x_test,y_test],x_test),
            callbacks=[checkpoint,earlystop])


In [ ]:
def plot(epochs,min_dim,hidden_layer_size,beta):
    global number_of_epochs
    loss_list = [s for s in history.history.keys() if 'recon_loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'recon_loss' in s and 'val' in s]
    number_of_epochs=len(history.history[loss_list[0]])
    epochs = range(1,number_of_epochs + 1)
    
    plt.figure()
    for l in loss_list:
        plt.plot(epochs, history.history[l])
    for l in val_loss_list:
        plt.plot(epochs, history.history[l])
    plt.title('Total loss extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs' )
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.yscale('log')
    plt.savefig('Total loss extended ' 
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    
    plt.figure()
    loss_list = [s for s in history.history.keys() if 'KL_loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'KL_loss' in s and 'val' in s]
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    plt.figure()
    for l in loss_list:
        plt.plot(epochs, history.history[l])
    for l in val_loss_list:
        plt.plot(epochs, history.history[l])
    plt.title('KL loss extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.yscale('log')
    plt.savefig('KL loss extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    
    plt.figure()
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s and 'KL' not in s and 'recon' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s and 'KL' not in s and 'recon' not in s]
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    plt.figure()
    for l in loss_list:
        plt.plot(epochs, history.history[l])
    for l in val_loss_list:
        plt.plot(epochs, history.history[l])
    plt.title('MSE loss extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.yscale('log')
    plt.savefig('MSE loss extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
              
    plt.figure()
    mse=[]
    errors=x_test[:100000]-cvae.predict([x_test[:100000],y_test[:100000]],batch_size=1000)
    for i in range(len(x_test[0,:])):
        plt.figure()
        plt.scatter(x_test[:1000,i],cvae.predict([x_test[:1000],y_test[:1000]],batch_size=1000)[:,i],alpha=1.0)
        plt.plot([0,0], [1,1], 'k-',color='w')
        error=errors[:,i]
        mse=np.append(mse,np.mean([x**2 for x in error])) 
        plt.xlabel(list(raw_data)[i+1]  + 'test values')
        plt.ylabel('prediction')
        plt.title('Predictions extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1] )
        
        plt.savefig('Predictions extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1] )
   
    plt.figure()
    plt.bar(range(len(mse)),mse,data=list(raw_data))
    plt.yscale('log')
    plt.title('MSE per variable extended ' 
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    plt.xlabel('variables')
    plt.ylabel('mse')
    plt.xticks(np.arange(19), list(raw_data)[1:20], rotation=30)
    plt.savefig('MSE per variable extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')

    latent_space_collection=pd.DataFrame(data=encoder.predict([x_test[:1000],y_test[:1000]],batch_size=1000))
    sns.pairplot(latent_space_collection).savefig('Pairplot extended ' 
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs')
    
    plt.show()

In [ ]:
def construct_numvec(digit, z=None):
    out1 = np.zeros((1,19))
    digit_one_hot = to_categorical(digit,2)
    if z is None:
        return np.concatenate([out1,digit_one_hot])
    else:
        return np.concatenate([z,digit_one_hot])

def generate_new(epochs,min_dim,hidden_layer_size,beta):
    digit = 1
    sample_numbers = 150000

    decoded_series=[]
    for i in range(0, sample_numbers):
        z=[np.mean(y_train[0,:]),np.mean(y_train[1,:]),np.mean(y_train[2,:])]
        for j in range(0,min_dim):        
            z.append(np.random.randn())
        vec = construct_numvec(digit,z)
        decoded = decoder.predict(np.reshape(vec,(1,min_dim+5)))
        decoded=np.reshape(decoded,(19,))
        decoded_series.append(decoded)
    decoded_series=np.asarray(decoded_series)


    data2=raw_data.values
    data2=np.delete(data2,0,axis=1)
    data2=np.delete(data2,len(data2[0,:])-1,axis=1)

    '''de-normalize'''
    for i in range(len(data2[0,:])):
        maximum=np.max(data2[:,i])
        minimum=np.min(data2[:,i])
        decoded_series[:,i]=np.multiply((maximum-minimum),decoded_series[:,i])+minimum
        
    np.save('decoded_series extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs ',
           decoded_series)
    for i in range(len(data[0,:])):
        plt.figure(i)
        plt.hist(decoded_series[:,i],200)
        plt.xlabel(list(raw_data)[i+1]  +' value')
        plt.ylabel('count')
        plt.title(' generated data historgram extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1] )
        
        plt.savefig(' generated data historgram extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1] )
        plt.show()

    for j in range(len(decoded_series[0,:])):
        for i in range(j+1,len(decoded_series[0,:])):
            plt.figure()
            plt.hist2d(decoded_series[:,i],decoded_series[:,j], (100, 100), norm=LogNorm(), cmap=plt.cm.jet)
            plt.title(list(raw_data)[i+1] + ' vs ' + list(raw_data)[j+1] +' generated valid points log density distribution')
            plt.xlabel(list(raw_data)[i+1])
            plt.ylabel(list(raw_data)[j+1])
            plt.colorbar()
            plt.savefig(list(raw_data)[i+1] + ' vs ' + list(raw_data)[j+1] +' generated valid points log density distribution')
            plt.show()
        
    digit = 0
    sample_numbers = 150000

    decoded_series=[]
    for i in range(0, sample_numbers):
        z=[np.mean(y_train[0,:]),np.mean(y_train[1,:]),np.mean(y_train[2,:])]
        for j in range(0,min_dim):        
            z.append(np.random.randn())
        vec = construct_numvec(digit,z)
        decoded = decoder.predict(np.reshape(vec,(1,min_dim+5)))
        decoded=np.reshape(decoded,(19,))
        decoded_series.append(decoded)
    decoded_series=np.asarray(decoded_series)


    data2=raw_data.values
    data2=np.delete(data2,0,axis=1)
    data2=np.delete(data2,len(data2[0,:])-1,axis=1)

    '''de-normalize'''
    for i in range(len(data2[0,:])):
        maximum=np.max(data2[:,i])
        minimum=np.min(data2[:,i])
        decoded_series[:,i]=np.multiply((maximum-minimum),decoded_series[:,i])+minimum
        
    np.save('decoded_series_false extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs',
           decoded_series)
    for i in range(len(data[0,:])):
        plt.figure(i)
        plt.hist(decoded_series[:,i],200)
        plt.xlabel(list(raw_data)[i+1]  +' value')
        plt.ylabel('count')
        plt.title(' generated data historgram false extended '
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1] )
        
        plt.savefig(' generated data historgram false extended ' 
                + str(min_dim) + ' dim latent ' 
                + str(hidden_layer_size) + ' neuron hidden ' 
                + str('%.0E' % beta) + ' beta ' 
                + str(number_of_epochs) + ' epochs'
                + list(raw_data)[i+1])
        
        plt.show()
        
    for j in range(len(decoded_series[0,:])):
        for i in range(j+1,len(decoded_series[0,:])):
            plt.figure()
            plt.hist2d(decoded_series[:,i],decoded_series[:,j], (100, 100), norm=LogNorm(), cmap=plt.cm.jet)
            plt.title(list(raw_data)[i+1] + ' vs ' + list(raw_data)[j+1] +' generated non valid points log density distribution')
            plt.xlabel(list(raw_data)[i+1])
            plt.ylabel(list(raw_data)[j+1])
            plt.colorbar()
            plt.savefig(list(raw_data)[i+1] + ' vs ' + list(raw_data)[j+1] +' generated non valid points log density distribution')
            plt.show()

In [ ]:
def do_all(epochs, min_dim, hidden_layer_size, beta):
    print(epochs, min_dim, hidden_layer_size, beta)
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    train(epochs, min_dim, hidden_layer_size, beta)
    plot(epochs, min_dim, hidden_layer_size, beta)
    generate_new(epochs, min_dim, hidden_layer_size, beta)

In [ ]:
epoch_list=[1000]
min_dim_list=[17]
hidden_layer_size_list=[54]
beta_list=[0.01]
for a in epoch_list:
    for b in min_dim_list:
        for c in hidden_layer_size_list:
            for d in beta_list:
                do_all(a,b,c,d)